In [12]:
import theano
from theano import tensor as T
import numpy as np
import sys
sys.path.append("../lib")
from load import mnist

Simple Compiled function example

In [2]:
a = T.scalar()
b = T.scalar()

y = a * b

multiply = theano.function(inputs=[a, b], outputs=y)

print multiply(1, 2) #2
print multiply(3, 3) #9

2.0
9.0


In [3]:
trX = np.linspace(-1, 1, 101)
trY = 2 * trX + np.random.randn(*trX.shape) * 0.33
print trY.shape

X = T.scalar()
print X
Y = T.scalar()
print Y


(101,)
<TensorType(float32, scalar)>
<TensorType(float32, scalar)>


In [14]:
def model1(X, w):
    return X * w

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def model2(X, w):
    return T.nnet.softmax(T.dot(X, w))

In [6]:
w = theano.shared(np.asarray(0., dtype=theano.config.floatX))
y = model1(X, w)

cost = T.mean(T.sqr(y - Y))
gradient = T.grad(cost=cost, wrt=w)
updates = [[w, w - gradient * 0.01]]

train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)


In [8]:
for i in range(100):
    for x, y in zip(trX, trY):
        train(x, y)
        
print w.get_value() #something around 2

1.8994102478


Logistic Regression

In [16]:
trX, teX, trY, teY = mnist(onehot=True)

X = T.fmatrix()
Y = T.fmatrix()

w = init_weights((784, 10))

py_x = model2(X, w)
y_pred = T.argmax(py_x, axis=1)

cost = T.mean(T.nnet.categorical_crossentropy(py_x, Y))
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * 0.05]]

In [17]:
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

In [18]:
for i in range(100):
    for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
        cost = train(trX[start:end], trY[start:end])
    print i, np.mean(np.argmax(teY, axis=1) == predict(teX))

0 0.8859
1 0.8976
2 0.9049
3 0.9085
4 0.9098
5 0.9112
6 0.9132
7 0.9141
8 0.9153
9 0.9156
10 0.9158
11 0.9165
12 0.917
13 0.9174
14 0.9178
15 0.9179
16 0.9181
17 0.9186
18 0.919
19 0.9195
20 0.9195
21 0.92
22 0.9204
23 0.9203
24 0.9203
25 0.9204
26 0.9205
27 0.9204
28 0.9205
29 0.9206
30 0.9209
31 0.921
32 0.9214
33 0.9213
34 0.9214
35 0.9216
36 0.9217
37 0.9219
38 0.9216
39 0.9217
40 0.9217
41 0.9218
42 0.9221
43 0.922
44 0.922
45 0.9222
46 0.9224
47 0.9224
48 0.9225
49 0.9229
50 0.9227
51 0.9228
52 0.9231
53 0.9231
54 0.9232
55 0.9231
56 0.9231
57 0.9231
58 0.9233
59 0.9235
60 0.9237
61 0.9237
62 0.9239
63 0.9241
64 0.9242
65 0.9241
66 0.9241
67 0.9239
68 0.924
69 0.924
70 0.9239
71 0.9239
72 0.9243
73 0.9243
74 0.9245
75 0.9245
76 0.9246
77 0.9243
78 0.9241
79 0.9241
80 0.9243
81 0.9243
82 0.9242
83 0.9244
84 0.9245
85 0.9245
86 0.9245
87 0.9244
88 0.9245
89 0.9245
90 0.9246
91 0.9246
92 0.9246
93 0.9249
94 0.925
95 0.9251
96 0.925
97 0.925
98 0.925
99 0.9247
